In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision import *
import os
from PIL import Image
from skimage import io, transform
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from skimage import io as sk_io

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
#ls "/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/"


In [ ]:
import numpy as np
import keras
class DataGenerator(keras.utils.Sequence):
    def __init__(self, root, valid_th=0, phase='train' , batch_size=33, dim=(152,152), n_channels=3 ,shuffle=True):
    
        folder_list = sorted(os.listdir(root))
        image_list=[]

        label_path='/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'# for labels
        label_list=list(np.array(sorted(os.listdir(label_path))))#[(train_pro[0]-1)])
        self.label_dict={label_list[i]:i for i in range(len(label_list))}

        for i in range(len(folder_list)):
            folder_pic=os.path.join(root,folder_list[i])
            list_pic = sorted(os.listdir(folder_pic))
            for j in range(len(list_pic)):
                img=os.path.join(folder_pic,list_pic[j])
                image_list.append(img)  

                    
        self.image_list = image_list
        self.root = root
        self.transform = transform
        self.batch_size = batch_size
        self.dim=dim
        self.n_channels=n_channels

    def __len__(self):
        return len(self.image_list)//self.batch_size
    
    
    def __data_generation(self, list_IDs_temp):
        
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            image = Image.open(list_IDs_temp[i]).convert('RGB').resize((152,152),Image.BICUBIC)
            image = np.array(image)
            image = image[None,:,:,:]
            X[i,] = image
    
            
            label = Path(list_IDs_temp[i]).stem
            num = str(int(label[-4:]))
            name = label[:-4]
            label = name+num
            y.append(label)
            
#             X=X.astype('float32'); X=X/255.0
#             y=y.astype('float32'); y=y/255.0
            #keras.utils.to_categorical(y, num_classes=self.n_classes)
        return X, y

    def __getitem__(self, idx):
        
        list_IDs_temp = self.image_list[idx*self.batch_size:(idx+1)*self.batch_size]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    def shuffle(self):
        import random
        random.shuffle(self.image_list)

In [ ]:
13233/33

In [ ]:
len(img_gen.image_list)
len(img_gen)
401*33

In [ ]:
import os
root='/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
img_gen = DataGenerator(root, phase='train')

In [ ]:
import os
from pathlib import Path
import keras
from keras.models import Model, Sequential
from keras.layers import Convolution2D, LocallyConnected2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import cv2
from keras.utils import plot_model
import numpy as np
import tensorflow as tf

In [ ]:

	base_model = Sequential()
	base_model.add(Convolution2D(32, (11, 11), activation='relu', name='C1', input_shape=(152, 152, 3)))
	base_model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='M2'))
	base_model.add(Convolution2D(16, (9, 9), activation='relu', name='C3'))
	base_model.add(LocallyConnected2D(16, (9, 9), activation='relu', name='L4'))
	base_model.add(LocallyConnected2D(16, (7, 7), strides=2, activation='relu', name='L5') )
	base_model.add(LocallyConnected2D(16, (5, 5), activation='relu', name='L6'))
	base_model.add(Flatten(name='F0'))
	base_model.add(Dense(4096, activation='relu', name='F7'))
	base_model.add(Dropout(rate=0.5, name='D0'))
	base_model.add(Dense(8631, activation='softmax', name='F8'))
    
	base_model.load_weights('/kaggle/input/my-dataset-for-test/VGGFace2_DeepFace_weights_val-0.9034.h5')
	#print(base_model.summary())

In [ ]:
deepface_model = Model(inputs=base_model.layers[0].input, outputs=base_model.layers[-3].output)
#print(deepface_model.summary())

In [ ]:
pip install tqdm

In [ ]:
from tqdm import tqdm

emb_list=[]
name_list=[]

for inc, data in tqdm(enumerate(img_gen)):
    img = data[0]
    label = data[1]
    emb = deepface_model.predict(img)
    #emb = emb.detach().numpy()
    label = list(label)
    for i in range(len(label)):
        emb_list.append(emb[i])
        name_list.append(label[i])


embeddings={}
for i in range(len(name_list)):
    embeddings[name_list[i]]=emb_list[i]


In [ ]:
df = pd.DataFrame(list(embeddings.items()),columns = ['name','emb']) 
df.to_pickle('pickle')

In [ ]:
import pandas as pd
df = pd.read_pickle('/kaggle/input/pretrained-models/pickle')
df

In [ ]:
embeddings = dict(zip(df.column1, df.column2))

embeddings.keys()

In [ ]:
def loadPairs(pairsFname):
    print("  + Reading pairs.")
    pairs = []
    with open(pairsFname, 'r') as f:
        for line in f.readlines()[1:]:
            #line = line[:-1]
            pair = line.strip().split(",")
            if len(pair[3]) == 0:
                del pair[3]
            #print(pair)
            pairs.append(pair)
    assert(len(pairs) == 6000)
    return np.array(pairs)

pair = loadPairs('/kaggle/input/lfw-dataset/pairs.csv')
pair.shape

In [ ]:
def getEmbeddings(pair, embeddings):
    if len(pair) == 3:
        name1 = "{}_{}".format(pair[0], pair[1])
        name2 = "{}_{}".format(pair[0], pair[2])
        actual_same = True
    elif len(pair) == 4:
        name1 = "{}_{}".format(pair[0], pair[1])
        name2 = "{}_{}".format(pair[2], pair[3])
        actual_same = False
    else:
        raise Exception(
            "Unexpected pair length: {}".format(len(pair)))

    (x1, x2) = (embeddings[name1], embeddings[name2])
    return (x1, x2, actual_same)

def getDistances(embeddings, pairs):
    list_dist = []
    y_true = []
    for pair in pairs:
        (x1, x2, actual_same) = getEmbeddings(pair, embeddings)
        diff = x1 - x2
        dist = np.dot(diff.T, diff)
        list_dist.append(dist)
        y_true.append(actual_same)
    return np.asarray(list_dist), np.array(y_true)

In [ ]:
import math
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
#from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score

def findBestThreshold(thresholds, embeddings, pairsTrain):
    bestThresh = bestThreshAcc = 0
    distances, y_true = getDistances(embeddings, pairsTrain)
    for threshold in thresholds:
        y_predlabels = np.zeros(y_true.shape)
        y_predlabels[np.where(distances < threshold)] = 1
        accuracy = accuracy_score(y_true, y_predlabels)
        if accuracy >= bestThreshAcc:
            bestThreshAcc = accuracy
            bestThresh = threshold
        else:
            # No further improvements.
            return bestThresh
    return bestThresh
thresholds = np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5])
bestThresh = findBestThreshold(thresholds, embeddings, pair)
bestThresh

In [ ]:
from sklearn.metrics import accuracy_score

def evalThresholdAccuracy(embeddings, pairs, threshold):
    distances, y_true = getDistances(embeddings, pairs)
    y_predict = np.zeros(y_true.shape)
    y_predict[np.where(distances < threshold)] = 1
    y_true = np.array(y_true)
    accuracy = accuracy_score(y_true, y_predict)
    return accuracy#, pairs[np.where(y_true != y_predict)]

acc = evalThresholdAccuracy(embeddings, pair, threshold=1.5)
acc

In [ ]:
def writeROC(fname, thresholds, embeddings, pairsTest):
    with open(fname, "w") as f:
        f.write("threshold,tp,tn,fp,fn,tpr,fpr\n")
        tp = tn = fp = fn = 0
        for threshold in thresholds:
            tp = tn = fp = fn = 0
            for pair in pairsTest:
                (x1, x2, actual_same) = getEmbeddings(pair, embeddings)
                diff = x1 - x2
                dist = np.dot(diff.T, diff)
                predict_same = dist < threshold

                if predict_same and actual_same:
                    tp += 1
                elif predict_same and not actual_same:
                    fp += 1
                elif not predict_same and not actual_same:
                    tn += 1
                elif not predict_same and actual_same:
                    fn += 1

            if tp + fn == 0:
                tpr = 0
            else:
                tpr = float(tp) / float(tp + fn)
            if fp + tn == 0:
                fpr = 0
            else:
                fpr = float(fp) / float(fp + tn)
            f.write(",".join([str(x)
                              for x in [threshold, tp, tn, fp, fn, tpr, fpr]]))
            f.write("\n")
            if tpr == 1.0 and fpr == 1.0:
                # No further improvements.
                f.write(",".join([str(x)
                                  for x in [4.0, tp, tn, fp, fn, tpr, fpr]]))
                return

In [ ]:
fname='roc_info'
writeROC(fname, thresholds, embeddings, pairsTest)

In [ ]:
rocData = pd.read_csv("roc_info")
fpr, tpr = rocData['fpr'], rocData['tpr']
plt.plot(fpr,tpr)